In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas as gpd # GeoPandas library for spatial analytics
from shapely.geometry import shape

import re

from urllib.parse import urlencode
import urllib.request, json 

import pyproj 

import matplotlib.pyplot as plt
import seaborn as sns # visualization styling package
%matplotlib inline 
import math
import pprint

# Read Json File
- remember to transform the csv file into json format through online tool, before reading it
https://www.convertcsv.com/csv-to-json.htm

In [3]:
#read json File
with open("../data/memories.json") as f:
    data = json.load(f)

data

[{'content': '“Within five years, I lost both my aunt and my mother to cancer. So, when I go to H Mart, I’m not just on the hunt for cuttlefish and three bunches of scallions for a buck; I’m searching for memories. I’m collecting the evidence that the Korean half of my identity didn’t die when they did. H Mart is the bridge that guides me away from the memories that haunt me, of chemo head and skeletal bodies and logging milligrams of hydrocodone.”\n\n— Crying in H Mart: A Memoir by Michelle Zauner\nhttps://a.co/1QSvs1R',
  'key_term': '',
  'concept': '',
  'ls': 'Adolescence',
  'Emotions': 'Acceptance, Nostalgic, Sadness',
  'Contributor': 'Michelle Zauner',
  'title': 'Crying in H Mart: A Memoir',
  'URL': '',
  'Created time': 'March 31, 2024 2:34 PM'},
 {'content': '“Within five years, I lost both my aunt and my mother to cancer. So, when I go to H Mart, I’m not just on the hunt for cuttlefish and three bunches of scallions for a buck; I’m searching for memories. I’m collecting t

In [8]:
# unique_records = []
# id_counter = 1

# for record in data:
#     ls_value = record.get('ls', '')  # Assuming 'ls' is the field containing comma-separated values
#     if ls_value:
#         ls_values = ls_value.split(',')  # Split the comma-separated values
#         for value in ls_values:
#             unique_record = {
#                 'id': id_counter,
#                 'ls': value.strip(),  # Remove leading/trailing whitespaces
#                 # Include other fields from the original record
#                 'content': record.get('content', None),
#                 'key_term': record.get('key_term', None),
#                 'concept': record.get('concept', None),
                
#                 'Contributor': record.get('Contributor', None),
#                 'title': record.get('title', None),
#                 'URL': record.get('URL', None),
#                 # create a new column value , and give them "1"
#                 'value': 1

#             }
#             unique_records.append(unique_record)
#             id_counter += 1

# unique_records

[{'id': 1,
  'ls': 'Adolescence',
  'content': '“Within five years, I lost both my aunt and my mother to cancer. So, when I go to H Mart, I’m not just on the hunt for cuttlefish and three bunches of scallions for a buck; I’m searching for memories. I’m collecting the evidence that the Korean half of my identity didn’t die when they did. H Mart is the bridge that guides me away from the memories that haunt me, of chemo head and skeletal bodies and logging milligrams of hydrocodone.”\n\n— Crying in H Mart: A Memoir by Michelle Zauner\nhttps://a.co/1QSvs1R',
  'key_term': '',
  'concept': '',
  'Contributor': 'Michelle Zauner',
  'title': 'Crying in H Mart: A Memoir',
  'URL': '',
  'value': 1},
 {'id': 2,
  'ls': 'Adolescence',
  'content': '“Within five years, I lost both my aunt and my mother to cancer. So, when I go to H Mart, I’m not just on the hunt for cuttlefish and three bunches of scallions for a buck; I’m searching for memories. I’m collecting the evidence that the Korean half 

In [10]:
unique_records = []
id_counter = 1

for record in data:
    ls_values = record.get('ls', '').split(',')
    emotions_values = record.get('Emotions', '').split(',')
    key_term_values = record.get('key_term', '').split(',')
    concept_values = record.get('concept', '').split(',')

    max_length = max(len(ls_values), len(emotions_values),
                     len(key_term_values), len(concept_values))

    for i in range(max_length):
        unique_record = {
            'id': id_counter,
            'content': record.get('content', "none"),
            'ls': ls_values[i].strip() if i < len(ls_values) and ls_values[i].strip() else "none",
            'key_term': key_term_values[i].strip() if i < len(key_term_values) and key_term_values[i].strip() else "none",
            'concept': concept_values[i].strip() if i < len(concept_values) and concept_values[i].strip() else "none",
            'Emotions': emotions_values[i].strip() if i < len(emotions_values) and emotions_values[i].strip() else "none",
            'Contributor': record.get('Contributor', "none"),
            'title': record.get('title', "none"),
            'URL': record.get('URL', "none"),
            'value': 1
        }
        unique_records.append(unique_record)
        id_counter += 1

unique_records

[{'id': 1,
  'content': '“Within five years, I lost both my aunt and my mother to cancer. So, when I go to H Mart, I’m not just on the hunt for cuttlefish and three bunches of scallions for a buck; I’m searching for memories. I’m collecting the evidence that the Korean half of my identity didn’t die when they did. H Mart is the bridge that guides me away from the memories that haunt me, of chemo head and skeletal bodies and logging milligrams of hydrocodone.”\n\n— Crying in H Mart: A Memoir by Michelle Zauner\nhttps://a.co/1QSvs1R',
  'ls': 'Adolescence',
  'key_term': 'none',
  'concept': 'none',
  'Emotions': 'Acceptance',
  'Contributor': 'Michelle Zauner',
  'title': 'Crying in H Mart: A Memoir',
  'URL': '',
  'value': 1},
 {'id': 2,
  'content': '“Within five years, I lost both my aunt and my mother to cancer. So, when I go to H Mart, I’m not just on the hunt for cuttlefish and three bunches of scallions for a buck; I’m searching for memories. I’m collecting the evidence that the

In [11]:
#Convert to json
with open("../data/grouped_memories_ls.json", "w") as outfile:
    json.dump(unique_records, outfile, indent=4)